In [1]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from scipy import spatial
import itertools

Using TensorFlow backend.


In [2]:

padded_embedding = pickle.load( open( "padded_embedding.p", "rb" ) )
#print(padded_embedding[1])


In [3]:
## randomize the list
random.shuffle(padded_embedding)

training_dataset,test_dataset = train_test_split(padded_embedding, train_size=.80, test_size=.20)



In [5]:
## dumping the train and test set


pickle.dump( training_dataset, open( "training_data.p", "wb" ) )

pickle.dump( test_dataset, open( "testing_data.p", "wb" ) )

In [12]:
## In this cell we segrigate inputs and labels for training the model
x, y = list(), list()
nonecount = list()
#print('First padding',a[0])
for k,i in enumerate(training_dataset):
    if isinstance(i,list):
        
        seq_x, seq_y = i[:len(i) - 1], i[-1]
        x.append(seq_x)
        y.append(seq_y)
    else:
        nonecount.append(k)
print(len(nonecount))

# convert the list to numpy array
num_x_train = np.array(x)
num_y_train = np.array(y)

0


In [16]:
## In this cell we segrigate inputs and labels for testing the model
x_test, y_test = list(), list()
nonecount1 = list()
#print('First padding',a[0])
for k,i in enumerate(test_dataset):
    if isinstance(i,list):
        
        seq_x_test, seq_y_test = i[:len(i) - 1], i[-1]
        x_test.append(seq_x_test)
        y_test.append(seq_y_test)
    else:
        nonecount.append(k)
print(len(nonecount1))

# convert the list to numpy array
num_x_test = np.array(x_test)
num_y_test = np.array(y_test)

0


In [25]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


output_embeddings = []

# choose a number of time steps
n_steps = 9

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 50
x = num_x_train.reshape((num_x_train.shape[0], num_x_train.shape[1], n_features))
y = num_y_train
#print(x)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(50))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = num_x_test
x_input = x_input.reshape((num_x_test.shape[0], n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
#print(yhat)
output_embeddings.append(yhat)


In [21]:
# univariate lstm example
## with sgd optimizer
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers

output_embeddings = []

# choose a number of time steps
n_steps = 9

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 50
x = num_x_train.reshape((num_x_train.shape[0], num_x_train.shape[1], n_features))
y = num_y_train
#print(x)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(50))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mse')
# fit model
model.fit(x, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = num_x_test
x_input = x_input.reshape((num_x_test.shape[0], n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
#print(yhat)
output_embeddings.append(yhat)

In [22]:
## dumping the sample outputs
pickle.dump( output_embeddings, open( "output_embeddings.p", "wb" ) )

In [23]:
## dumping the original outputs
pickle.dump( num_y_test, open( "original_output_embeddings.p", "wb" ) )

In [28]:
# for efficient memory
del padded_embedding
del training_dataset,test_dataset
del num_x_train
del num_y_train
del num_x_test

del x_test
del y_test

In [29]:
# Load the overall embeddings received from Pyke

entire_embeddings = pickle.load( open( "Pyke50_pickle.p", "rb" ) )

In [30]:
# Creating a dictionary with only the item embeddings


# dictionary of item embeddings
item_embedddings = {} 
for key, val in entire_embeddings.items(): 
    if key.startswith('http://hm2.com/article#'):
        
        
        item_embedddings.update( {key : val} )
          
print(len(item_embedddings))
pickle.dump( item_embedddings, open( "item_embedddings.p", "wb" ) )

151156


In [32]:
## convert output embedings into 1 -D

new_output_embeddings = list(itertools.chain.from_iterable(output_embeddings))

In [ ]:
## this cell calculates the cosine similarity among y and y_hats
final_cosine = []


for yhat in new_output_embeddings:
    local_cosine = []
    for k,v in item_embedddings.items():

        result = 1 - spatial.distance.cosine(yhat, v)
        local_cosine.append(result)
    final_cosine.append(local_cosine)

In [14]:
## this cell ranks the urls in decending order and lists the top n most frequent elemnts
ranks = []
for i in final_cosine:
    numpy_cosine = np.array(i)
    rank_list = numpy_cosine.argsort()[::-1][:50] ## take top 50 prediction for each item
    ranks.append(rank_list)

#ranks

[array([14,  8,  7, 11,  9, 10, 12, 13,  6,  1]),
 array([13,  2,  3,  7, 12,  8,  6,  1, 11,  4]),
 array([14,  8,  7, 11,  9, 10, 12, 13,  6,  1])]

In [15]:
# this cell captures y_hat predictions
y_hat = []
for lists in ranks:
    local_key = []
    for key in lists:
        first_key = list(item_embedddings)[key]
        local_key.append(first_key)
    y_hat.append(local_key)
#y_hat

In [16]:
## load original keys of y
y = []
original_output_embeddings = num_y_test.tolist()
#print(len(original_sample_output_embeddings))
for i in original_output_embeddings:
    for k,v in item_embedddings.items():
        if i ==v:
            y.append(k)


    

In [17]:
## This cell is to generate the matrix for final learning
## so if original label is present in top n prediction labels , increase the count by 1
y_arr = [None]* len(y)
y_hat_arr = [None]* len(y_hat)
y_arr = y
y_hat_arr = y_hat

i = 0
count = 0
while(i<len(y_arr)):
    if (y_arr[i] in y_hat_arr[i]):
        count +=1
    i +=1
Ratio = count/len(y_arr)
print('Learning Ratio is : ',Ratio)

Learning Ratio is :  0.0


In [4]:
final_cosine = [[0.9999998221032514, 0.9999998823736738, 0.9999999174768217, 0.999999898314237, 0.9999998957033124], [0.9999998543873563, 0.9999999146544587, 0.99999994975972, 0.9999999305959917, 0.9999999279861516]]
ranks = []
for i in final_cosine:
    numpy_cosine = np.array(i)
    rank_list = numpy_cosine.argsort()[::-1][:7] ## take top 50 prediction for each item
    ranks.append(rank_list)
ranks

[array([2, 3, 4, 1, 0]), array([2, 3, 4, 1, 0])]